In [225]:
#Реализовать метод решения СЛАУ, на выбор: метод вращений или метод отражений.
#Вычислить числа обусловленности. Протестировать на тех же матрицах, что использовались в задании 2; сравнить.

Метод вращений

In [226]:
import pandas as pd
import numpy as np
import math

In [227]:
def generate_tridiagonal_matrix(n: int, digits: int = -1) -> np.array:
    matrix = np.empty(shape=(n, n))
    for i in range(n):
        for j in range(n):
            if j == i - 1 or j == i + 1:
                current_value = -1
            elif j == i:
                current_value = 2
            else:
                current_value = 0
            if digits != -1:
                if digits <= 0:
                    raise AttributeError("digits must be positive value")
                current_value = round(current_value, digits)
            matrix[i, j] = current_value
    return matrix

In [228]:
#Число обусловленности
def number_of_conditionality(matrix: np.ndarray) -> float:
    return np.linalg.norm(matrix) * np.linalg.norm(np.linalg.inv(matrix))

In [229]:
#Создаем матрицу поворота
def create_rotational_matrix(n, sin, cos, i, j) -> np.array:
    matrix = np.eye(n)
    matrix[i, i] = cos
    matrix[i, j] = -sin
    matrix[j, i] = sin
    matrix[j, j] = cos
    return matrix

In [230]:
#Заполняем матрицу нужными эл-тами
def get_sin_cos(a, b):
    if a == 0 and b == 0:
        return 0, 1
    r = math.sqrt(a ** 2 + b ** 2)
    return -(b / r), a / r

In [231]:
def qr_decompose(A: np.array):
    n = A.shape[0]

    Q = np.eye(n)
    R = A.copy()
    for j in range(n):
        for i in range(n - 1, j, -1):
            a, b = R[i - 1, j], R[i, j]
            sin, cos = get_sin_cos(a, b)
            rotational_matrix = create_rotational_matrix(n, sin, cos, i, j)
            R = rotational_matrix.T.dot(R)
            Q = Q.dot(rotational_matrix)
    return Q, R


In [232]:
# to solve Ux = b
def back_sub(U, b):
    n = U.shape[0]
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        tmp = b[i]
        for j in range(i + 1, n):
            tmp -= U[i, j] * x[j]
        x[i] = tmp / U[i, i]
    return x


In [233]:
# Rx = Q_tb
def qr_solve(A, b):
    Q, R = qr_decompose(A)

    print(f"Condition A: {number_of_conditionality(A)}")
    print(f"Condition Q: {number_of_conditionality(Q)}")
    print(f"Condition R: {number_of_conditionality(R)}")

    y = Q.T.dot(b)
    x = back_sub(R, y)
    return x

In [234]:
A = np.array([[6, 5, 0],
              [5, 1, 4],
              [0, 4, 3]])
b = np.array([1, 1, 1])

actual = qr_solve(A,b)

expected = np.linalg.solve(A, b)

print(np.linalg.norm(actual - expected))

Condition A: 4.219449098422382
Condition Q: 3.0
Condition R: 4.219449098422382
1.3877787807814457e-17


In [235]:
A = np.array([[1, 1/2, 1/3, 1/4, 1/5],
              [1/2, 1/3, 1/4, 1/5, 1/6],
              [1/3, 1/4, 1/5, 1/6, 1/7],
              [1/4, 1/5, 1/6, 1/7, 1/8],
              [1/5, 1/6, 1/7, 1/8, 1/9]])
b = np.array([1, 1, 1, 1, 1])

actual = qr_solve(A,b)

expected = np.linalg.solve(A, b)

print(np.linalg.norm(actual - expected))

Condition A: 480849.11699433636
Condition Q: 5.000000000000001
Condition R: 480849.1169940533
1275.870895539019


In [236]:
A = np.array([[1, 1/2, 1/3, 1/4, 1/5, 1/6, 1/7],
              [1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/8],
              [1/3, 1/4, 1/5, 1/6, 1/7, 1/8, 1/9],
              [1/4, 1/5, 1/6, 1/7, 1/8, 1/9, 1/10],
              [1/5, 1/6, 1/7, 1/8, 1/9, 1/10, 1/11],
              [1/6, 1/7, 1/8, 1/9, 1/10, 1/11, 1/12],
              [1/7, 1/8, 1/9, 1/10, 1/11, 1/12, 1/13]])
b = np.array([1, 1, 1, 1, 1, 1, 1])

actual = qr_solve(A,b)

expected = np.linalg.solve(A, b)

print(np.linalg.norm(actual - expected))

Condition A: 481747252.56394905
Condition Q: 7.000000000000001
Condition R: 481747254.9448719
52353.75993546792


In [237]:
A=generate_tridiagonal_matrix(5) 
b=np.array([1, 2, 1, 2, 1])

actual = qr_solve(A,b)

expected = np.linalg.solve(A, b)

print(np.linalg.norm(actual - expected))

Condition A: 20.739120307069708
Condition Q: 5.000000000000001
Condition R: 20.73912030706971
2.6645352591003757e-15
